## Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import datasets as ds
from collections import Counter
import re
from bs4 import BeautifulSoup

/Users/francois.soulier/miniconda/envs/SCIA/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


## The dataset

### Question 1
How many splits does the dataset has?

In [4]:
splits: list[str] = ds.get_dataset_split_names('imdb')
print(splits)
print(f'Number of splits: {len(splits)}')

['train', 'test', 'unsupervised']
Number of splits: 3


There are 3 splits in the IMDB dataset.

### Question 2
How big are these splits?

In [5]:
def load_datasets() -> list[ds.Dataset]:
    """
    Loads the IMDB dataset from the datasets library.
    Returns:
        datasets: list[ds.Dataset] - List of datasets
    """
    datasets: list[ds.Dataset] = []
    for split in splits:
        dataset: ds.Dataset = ds.load_dataset('imdb', split=split)
        datasets.append(dataset)
    
    return datasets

datasets: list[ds.Dataset] = load_datasets()
for i, dataset in enumerate(datasets):
    print(f'{splits[i]} split size : {dataset.num_rows}')

Found cached dataset imdb (/Users/francois.soulier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/Users/francois.soulier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/Users/francois.soulier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


train split size : 25000
test split size : 25000
unsupervised split size : 50000


### Question 3
What is the proportion of each class on the supervised splits?

In [6]:
# Get only supervised datasets
supervised_datasets: list[pd.DataFrame] = list(map(lambda x: x.to_pandas(), datasets[0:2]))

# For each dataset, print the number of samples for each class
for i, dataset in enumerate(supervised_datasets):
    data_frame = dataset
    print(splits[i])
    print('Class 0')
    print(data_frame.where(data_frame['label'] == 0).count())
    print('Class 1')
    print(data_frame.where(data_frame['label'] == 1).count())
    print('\n')

train
Class 0
text     12500
label    12500
dtype: int64
Class 1
text     12500
label    12500
dtype: int64


test
Class 0
text     12500
label    12500
dtype: int64
Class 1
text     12500
label    12500
dtype: int64




Hence, each class represents 50% of the supervised dataset (both in train and test samples).

## Naive Bayes classifier 

### Question 1
Create an adapted processing function which lower case the text and replace punctuations with text:

In [7]:
def clean_html(text: str) -> str:
  """
  Removes HTML tags from the given text.
  Args:
      text (str): Text with html tags.
  Returns:
      str: Text from all html tags.
  """
  no_html = BeautifulSoup(text).get_text()
  return no_html

def text_processing(text: str) -> str:
  """
  Pre-processes the given text.
  Args:
      text (str): Text to process
  Returns:
      str: Processed text
  """
  result_text = text
  result_text = clean_html(result_text)
  result_text = result_text.lower()
  pattern = r"(?<![a-zA-Z])[^\w\s]|[^\w\s](?![a-zA-Z])"
  result_text = re.sub(pattern, "", result_text)
  result_text = result_text.strip()
  return re.sub("(\s+)", " ", result_text)

#### Tiny test (preprocessing)

In [8]:
def test_preprocessing(input: str, expected: str) -> None:
    result: str = text_processing(input)
    assert text_processing(input) == expected or print(result)

test_preprocessing("Hello, ,,,World!::", "hello world")
test_preprocessing("Hello,        U.S.A!", "hello u.s.a")

Now let's apply `text_processing` function on `text` field of our training dataset.

In [9]:
train_data_frame: pd.DataFrame = supervised_datasets[0]
train_data_frame.text = train_data_frame.text.apply(text_processing)

/Users/francois.soulier/miniconda/envs/SCIA/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


### Tokenize function
Function to cut processed text into tokens.

In [10]:
def tokenize(text: str)-> list:
    """
    Tokenizes the given text.
    Args:
        text (str): Text to tokenize (pre-processed)
    Returns:
        list: List of tokens
    """
    return [w for w in re.split("\W+", text)]

### Create Dataframe for vocabulary
This function compute the vocabulary dataset.

In [11]:
def build_vocabulary(texts_serie: pd.Series) -> Counter:
    """
    Builds the vocabulary of the given texts serie.
    Args:
        text_serie (pd.Series): Text serie
    Returns:
        Counter: Vocabulary
    """
    vocabulary: Counter = None # Use Counter as a dictionary with word occurrences
    for text in texts_serie:
        word_list: list[str] = tokenize(text=text)
        if vocabulary is None:
            vocabulary = Counter(word_list)
        else:
            vocabulary.update(word_list)
    return vocabulary

Let's compute the vocabulary and change the label type of the train data frame.

In [12]:
vocabulary: Counter = build_vocabulary(texts_serie=train_data_frame.text)
train_data_frame.label = train_data_frame.label.astype(str)
counter_class: pd.DataFrame = train_data_frame.groupby("label").agg({'text': build_vocabulary})

Function which returns occurence of a word of a specific class.

In [13]:
def word_count(counter_class: pd.DataFrame, class_name: str, word: str) -> int:
    """
    Returns the number of occurrences of the given word in the given class.
    Args:
        counter_class (pd.DataFrame): DataFrame with the vocabulary of each class
        class_name (str): Class name / label
        word (str): Word
    Returns:
        int: Number of occurrences of the given word in the given class
    """
    return counter_class.loc[class_name]["text"][word]

## Implementation of naive Bayes classifier

Let's implement this pseudo-code:

![Alt text](nbc.png)

In [14]:
def total_words(vocabulary: Counter, c: str, counter_class: pd.DataFrame) -> int:
    """
    Returns the total number of words in a class for the given vocabulary.
    Args:
        vocabulary (Counter): Vocabulary
        c (str): Class name / label
        counter_class (pd.DataFrame): DataFrame with the vocabulary of each class
    Returns:
        int: Total number of words in the given class
    """
    total: int = 0
    for w in vocabulary:
        total += word_count(counter_class, c, w)
    return total

Function which affect a specified value for loglikelihood dictionnary at index `word,class_value`. It represent the loglikelihood for a word of a specific class.

In [15]:
def fill_loglikelihood(loglikelihood: dict, word: str, class_value: str, value_to_affect: float) -> None:
    """
    Fills the loglikelihood dictionary with the given values.
    Args:
        loglikelihood (dict): Loglikelihood dictionary
        word (str): Word
        class_value (str): Class name / label
        value_to_affect (float): Value to affect
    """
    if (loglikelihood.get(word) is None):
        loglikelihood[word] = {}
    loglikelihood[word][class_value] = value_to_affect

In [16]:
def naive_bayes_classifier(train_data_frame: pd.DataFrame, vocabulary: Counter, counter_class: pd.DataFrame) -> tuple[dict, dict, Counter]:
    """
    Builds the Naive Bayes classifier.
    Args:
        train_data_frame (pd.DataFrame): Training data frame
        vocabulary (Counter): Vocabulary
        counter_class (pd.DataFrame): DataFrame with the vocabulary of each class
    Returns:
        tuple[dict, dict, Counter]: Tuple with the logprior, loglikelihood and vocabulary
    """
    total_document_count: int = train_data_frame.text.count()
    class_label_set: list = list(train_data_frame.groupby("label").groups.keys())
    logprior: dict = {}
    loglikelihood: dict = {}
    
    for current_class in class_label_set:
        class_document_count: int = train_data_frame[train_data_frame.label == current_class].text.count()
        logprior[current_class] = np.log(class_document_count/total_document_count)
        total: int = total_words(vocabulary,current_class,counter_class) + len(vocabulary)
        
        for word in vocabulary:
            count_w_c = word_count(counter_class, current_class, word) + 1
            log_like_value = np.log(count_w_c / total)
            fill_loglikelihood(loglikelihood,word,current_class,log_like_value)
            
    return logprior, loglikelihood, vocabulary

## Test Naive Bayes classifier

Here is the test function of our naive Bayes classifier which apply for one testdoc.

In [17]:
def test_naive_bayes_classifier(testdoc: str, logprior: dict, loglikelihood: dict, train_data_frame: pd.DataFrame, vocabulary: Counter) -> tuple:
    """
    Tests the Naive Bayes classifier.
    Args:
        testdoc (str): Test document
        logprior (dict): Logprior
        loglikelihood (dict): Loglikelihood
        train_data_frame (pd.DataFrame): Training data frame
        vocabulary (Counter): Vocabulary
    Returns:
        tuple: Tuple with the predicted class and the loglikelihood
    """
    class_set: list = list(train_data_frame.groupby("label").groups.keys())
    sums: dict = {}
    max_class = None

    for c in class_set:
        sums[c] = logprior[c]
        word_list = tokenize(testdoc)
        for w in word_list:
            if(vocabulary[w] != 0):
                sums[c] = sums[c] + loglikelihood[w][c]

        if (max_class is None or sums[max_class] < sums[c]):
            max_class = c
        
    return max_class

### Process data
Make prediction on each text from `test_data_frame` and store them in `model_result`. 

In [18]:
test_data_frame: pd.DataFrame = supervised_datasets[1]
test_data_frame.label = test_data_frame.label.astype(str)
logprior, loglikelihood, vocabulary = naive_bayes_classifier(train_data_frame,vocabulary,counter_class)
test_data_frame_2: pd.DataFrame = test_data_frame
test_nbc_function = lambda text : test_naive_bayes_classifier(text, logprior,loglikelihood,train_data_frame,vocabulary)
test_data_frame_2["model_result"] = test_data_frame_2.text.apply(test_nbc_function)

### Get results
We are now able to get the good predictions count, hence we can get an accuracy ratio.

In [26]:
good_predictions_count: int = (test_data_frame_2[test_data_frame_2.label == test_data_frame_2.model_result]).label.count()
text_count: int = test_data_frame_2.text.count()
accuracy_ratio: float = good_predictions_count / text_count
print(f'Test accuracy : {(accuracy_ratio * 100):.2f}')

Test accuracy : 82.40
